In [4]:
from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from src.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Base.Evaluation.Evaluator import EvaluatorHoldout
from src.Utils.load_URM import load_URM
from src.Utils.load_ICM import load_ICM
import numpy as np

URM_all = load_URM("in/data_train.csv")
ICM_all = load_ICM("in/data_ICM_title_abstract.csv")
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [9]:
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [6]:
MAP=[]

recommender = P3alphaRecommender(URM_train)

for topK in [10, 50, 100, 200, 500, 600]:
    for alpha in np.linspace(start=0.0,stop=1.0,num=15):
        recommender.fit(topK=topK,alpha=alpha, implicit=True)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append({'topK':topK,'alpha':alpha,'MAP':result_dict[10]["MAP"]})
        print(MAP[-1])

P3alphaRecommender: URM Detected 71 (0.89 %) cold users.
P3alphaRecommender: URM Detected 2382 (9.17 %) cold items.
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 3.01 sec. Users per second: 1867
{'topK': 10, 'alpha': 0.0, 'MAP': 0.03937037661546498}
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 2.93 sec. Users per second: 1919
{'topK': 10, 'alpha': 0.07142857142857142, 'MAP': 0.04465256653017886}
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 2.94 sec. Users per second: 1917
{'topK': 10, 'alpha': 0.14285714285714285, 'MAP': 0.04595969372869834}
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 2.99 sec. Users per second: 1879
{'topK': 10, 'alpha': 0.21428571428571427, 'MAP': 0.04729619967640107}
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 2.99 sec. Users per second: 1884
{'topK': 10, 'alpha': 0.2857142857142857, 'MAP': 0.04869267761429508}
EvaluatorHoldout: Processed 5628 ( 100.00% ) in 3.01 sec. Users per second: 1872
{'topK': 10, 'alpha': 0.3571428571428571, 'MAP': 0.048981914

In [7]:
max_tuple = max(MAP, key= lambda x: x['MAP'])
print('The best combination for the P3alpha recommender is ({},{}), with a MAP = {}'.format(max_tuple['topK'],max_tuple['alpha'], max_tuple['MAP']))

The best combination for the P3alpha recommender is (200,0.5), with a MAP = 0.05719859810945092


In [10]:
Similarity_1 = P3alphaRecommender(URM_train=URM_train)
Similarity_1.fit(topK=200, alpha=0.5, implicit=True)
Similarity_1 = Similarity_1.W_sparse
Similarity_2 = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM_all)
Similarity_2.fit(
    topK=100,
    shrink=10,
    similarity='jaccard')
Similarity_2 = Similarity_2.W_sparse


MAP=[]

for topK in [10, 50, 100, 200, 500]:
    for alpha in np.linspace(start=0.0,stop=1.0,num=15):
        recommender = ItemKNNSimilarityHybridRecommender(URM_train=URM_train, Similarity_1=Similarity_1, Similarity_2=Similarity_2)
        recommender.fit(topK=topK, alpha=alpha)

        result_dict, _ = evaluator_validation.evaluateRecommender(recommender)

        MAP.append({'topK':topK,'alpha':alpha,'MAP':result_dict[10]["MAP"]})
        print(MAP[-1])

P3alphaRecommender: URM Detected 67 (0.84 %) cold users.
P3alphaRecommender: URM Detected 2369 (9.12 %) cold items.
ItemKNNCBFRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNCBFRecommender: URM Detected 2369 (9.12 %) cold items.
Similarity column 25975 ( 100 % ), 4443.35 column/sec, elapsed time 0.10 min
ItemKNNSimilarityHybridRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2369 (9.12 %) cold items.
EvaluatorHoldout: Processed 5597 ( 100.00% ) in 3.02 sec. Users per second: 1854
{'topK': 10, 'alpha': 0.0, 'MAP': 0.03242366807561172}
ItemKNNSimilarityHybridRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNSimilarityHybridRecommender: URM Detected 2369 (9.12 %) cold items.
EvaluatorHoldout: Processed 5597 ( 100.00% ) in 3.05 sec. Users per second: 1835
{'topK': 10, 'alpha': 0.07142857142857142, 'MAP': 0.0370774952246265}
ItemKNNSimilarityHybridRecommender: URM Detected 67 (0.84 %) cold users.
ItemKNNSimilarityHybridRecomm

In [11]:
max_tuple = max(MAP, key=lambda x: x['MAP'])
print('The best combination for the hybrid recommender is ({},{}), with a MAP = {}'.format(max_tuple['topK'],max_tuple['alpha'], max_tuple['MAP']))

The best combination for the hybrid recommender is (500,0.42857142857142855), with a MAP = 0.07006728068248053
